# Behavioural cloning project

In [37]:
# Configure matlab to show graphics in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [38]:
# Change to 'data' when training on a GPU
PATH_TO_DATA = 'data'

In [39]:
# Shape of the image
input_shape = (66, 200, 3)

In [40]:
def img_folder(data_folder):
    return '{}/IMG'.format(data_folder)

def path_driving_log(data_folder):
    return '{}/driving_log.csv'.format(data_folder)

In [41]:
datasets = ['train', 'test', 'valid']

In [42]:
def get_driving_log_dataframe(data_folder):
    driving_log_df = pd.read_csv(path_driving_log(data_folder))
    return driving_log_df

In [43]:
path_to_folders = dict(zip(datasets, map(lambda folder: '{0}/{1}'.format(PATH_TO_DATA, folder), datasets)))

In [44]:
path_to_folders

{'test': 'data/test', 'train': 'data/train', 'valid': 'data/valid'}

In [45]:
!pip install imutils

In [46]:
import imutils

In [59]:
from scipy.ndimage import imread
from os import listdir
from sklearn.utils import shuffle

from PIL import Image

def image_label_generator(data_folder, batch_size=64):
    driving_log_df = get_driving_log_dataframe(data_folder)
    number_of_examples = len(driving_log_df)
    image_columns = ['center', 'left', 'right']
    
    X_train = []
    y_train = []
    weights = []
    index_in_batch = 0
    batch_number = 0
    
    while True:
        for image_column in image_columns:
            image_series = driving_log_df[image_column]
            steering_series = driving_log_df['steering']
            for offset in range(0, number_of_examples, batch_size):
                X_train = []
                y_train = []
                weights = []

                end_of_batch = min(number_of_examples, offset + batch_size)

                for j in range(offset, end_of_batch):
                    image_filename = image_series[j].lstrip().rstrip()
                    image = Image.open('{0}/{1}'.format(data_folder, image_filename))
                    image = np.asarray(image.resize((200, 66)))
                    label = steering_series[j]
                    
                    if abs(label) < 10e-4:
                        weights.append(1)
                    else:
                        weights.append(10e2)
                    
                    X_train.append(image)
                    y_train.append(label)
                    X_train, y_train, weights = shuffle(X_train, y_train, weights)

                yield np.array(X_train), np.array(y_train), np.array(weights)

In [72]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import Flatten

In [73]:
model = Sequential()

In [74]:
model.add(Convolution2D(3, 1, 1, 
                          input_shape=input_shape, 
                          border_mode='same', 
                          activation='relu',
                          init='he_normal'))
model.add(BatchNormalization())
model.add(Convolution2D(24, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Convolution2D(36, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Convolution2D(48, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Convolution2D(64,
                       3, 3,
                       init='he_normal'))
model.add(Convolution2D(64,
                       3, 3,
                       init='he_normal'))
model.add(Flatten())
model.add(Dense(100, activation='relu', init='he_normal'))
model.add(Dense(50, activation='relu', init='he_normal'))
model.add(Dense(10, activation='relu', init='he_normal'))
model.add(Dense(1, activation='linear', init='he_normal'))

In [75]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_25 (Convolution2D) (None, 66, 200, 3)    12          convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
batchnormalization_5 (BatchNorma (None, 66, 200, 3)    12          convolution2d_25[0][0]           
____________________________________________________________________________________________________
convolution2d_26 (Convolution2D) (None, 31, 98, 24)    1824        batchnormalization_5[0][0]       
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 14, 47, 36)    21636       convolution2d_26[0][0]           
___________________________________________________________________________________________

In [76]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath)
callbacks_list = [checkpoint]

In [77]:
from keras.optimizers import Adam
adam = Adam(lr=10e-4)

In [78]:
model.compile(optimizer=adam, 
              loss='mse',
             metrics=['mean_squared_error'])

In [79]:
image_generator = image_label_generator(PATH_TO_DATA)
samples = len(get_driving_log_dataframe(PATH_TO_DATA)) * 3

In [80]:
model.fit_generator(image_generator, 
                    samples_per_epoch=samples, 
                    nb_epoch=5,
                   callbacks=callbacks_list)

Epoch 1/5
24108/24108 [==============================] - 68s - loss: 15.3772 - mean_squared_error: 0.0196    
Epoch 2/5
24108/24108 [==============================] - 67s - loss: 12.8071 - mean_squared_error: 0.0177    
Epoch 3/5
24108/24108 [==============================] - 67s - loss: 10.6237 - mean_squared_error: 0.0149    
Epoch 4/5
24108/24108 [==============================] - 67s - loss: 9.5721 - mean_squared_error: 0.0146    
Epoch 5/5
24108/24108 [==============================] - 67s - loss: 10.3443 - mean_squared_error: 0.0157    


In [81]:
X_val, y_val, w = next(image_label_generator(PATH_TO_DATA))

In [82]:
model.predict(X_val)

array([[ 0.23010135],
       [-0.31275859],
       [ 0.23177318],
       [ 0.22807331],
       [ 0.18788563],
       [ 0.25141466],
       [ 0.24830188],
       [ 0.22650208],
       [ 0.24438733],
       [-0.3199549 ],
       [-0.3263177 ],
       [-0.34536847],
       [-0.3281562 ],
       [ 0.25063214],
       [-0.31791669],
       [-0.29532936],
       [ 0.19988415],
       [ 0.21582446],
       [-0.32666871],
       [-0.34208867],
       [ 0.23997025],
       [-0.32569155],
       [-0.3280814 ],
       [ 0.21360563],
       [ 0.24851169],
       [-0.34297889],
       [ 0.23452841],
       [-0.30654326],
       [-0.29814893],
       [ 0.19857794],
       [ 0.25649378],
       [ 0.22667091],
       [ 0.19387136],
       [-0.3357884 ],
       [-0.33707079],
       [ 0.2288669 ],
       [-0.31611565],
       [-0.323504  ],
       [-0.32570142],
       [-0.3295522 ],
       [-0.31799823],
       [ 0.25839666],
       [-0.32413092],
       [-0.33832431],
       [ 0.23297074],
       [ 0

In [83]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [39]:
y_val

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.0617599 ,
        0.        ,  0.        ,  0.3679529 ,  0.        ,  0.5784606 ,
        0.        ,  0.        ,  0.5784606 ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.05219137,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.1670138 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.05219137,  0.        ])